# 3주차 스터디

3주차 스터디는 pandas 라이브러리 기초문법을 익히면서, 데이터를 자유자재로 다루는 방법에 대해 알아보겠습니다. <br>

#### 2주차 스터디의 목차
- 데이터 병합(merge, concat)
- 계층(다중) 색인
- 통계함수와 데이터 정렬하기
- 집계(그룹화) 함수 : groupby, pivot_table

## 패키지 불러오기

In [2]:
import pandas as pd
import numpy as np

# 1. 데이터 병합(merge, concat)

데이터 분석을 하다보면, 내가 필요한 데이터를 찾아와 추가할 때가 많다. 이 데이터를 추가하는 방법에 대해 알아보고자 한다. <br>

데이터를 병합(합치기)하는 대표적인 함수는 merge, concat가 있다. <br>

merge는 보통 공통되는 열, 행을 기준으로 잡고 합칠 때 많이 쓰이고<br>
concat은 공통되지 않고 무작정 데이터를 합쳐보고자 할 때 많이 쓰인다.

## merge 함수

#### pandas.merge()
<img src="img/merge예제.jpg" alt="merge예제" style="width: 600px;"/>

In [46]:
# sample data 생성
df1 = pd.DataFrame({'A' : [1, 1, 2],
                   'B' : [3, 2, 5]})

df2 = pd.DataFrame({'A' : [1, 2, 3],
                   'C' : ['Big', 'Data', 'analytics']})

In [47]:
df1

,A,B
0,1,3
1,1,2
2,2,5


In [48]:
df2

,A,C
0,1,Big
1,2,Data
2,3,analytics


In [49]:
# 'A'를 기준으로 합치기
# pd.merge(df1, df2, on = 'A')
pd.merge(df1, df2, on = 'A')

,A,B,C
0,1,3,Big
1,1,2,Big
2,2,5,Data


### 주요 파라미터
키를 기준으로 DataFrame의 로우를 합친다. SQL이나 다른 관계형 데이터베이스의 join 연산과 동일하다 <br>

   - 주요 파라미터        
      . left, right : merge할 DataFrame 객체이름
      
      . how = 'inner', #left, right, outer
      
      . on = None, #merge의 기준이 되는 컬럼
      
      . left_on = None, #left DataFrame의 기준 컬럼
      
      . right_on = None, #right DataFrame의 기준 컬럼

In [50]:
df_left = pd.DataFrame({'KEY': ['k0', 'k1', 'k2', 'k3'],
                     'A': ['a0', 'a1', 'a2', 'a3'],
                     'B': ['b0', 'b1', 'b2', 'b3']})
df_right = pd.DataFrame({'KEY': ['k2', 'k3', 'k4', 'k5'],
                     'C': ['c2', 'c3', 'c4', 'c5'],
                     'D': ['d2', 'd3', 'd4', 'd5']})

In [51]:
# df_left 출력
df_left

,KEY,A,B
0,k0,a0,b0
1,k1,a1,b1
2,k2,a2,b2
3,k3,a3,b3


In [52]:
# df_right 출력
df_right

,KEY,C,D
0,k2,c2,d2
1,k3,c3,d3
2,k4,c4,d4
3,k5,c5,d5


In [55]:
# inner join : 공통된 것만 불러와서 합친다.
# on에 기준 열을 안 써줬지만 공통인 열은 key밖에 없으므로 알아서 합쳐짐
pd.merge(df_left, df_right, how='inner')

,KEY,A,B,C,D
0,k2,a2,b2,c2,d2
1,k3,a3,b3,c3,d3


In [56]:
# left outer join
pd.merge(df_left, df_right, how="left")

,KEY,A,B,C,D
0,k0,a0,b0,NaN,NaN
1,k1,a1,b1,NaN,NaN
2,k2,a2,b2,c2,d2
3,k3,a3,b3,c3,d3


In [57]:
# right outer join
pd.merge(df_left,df_right, how = 'right')

,KEY,A,B,C,D
0,k2,a2,b2,c2,d2
1,k3,a3,b3,c3,d3
2,k4,NaN,NaN,c4,d4
3,k5,NaN,NaN,c5,d5


In [59]:
# fully outer join : 일단 있는 값들은 다 불러놓고, 없는 값들은 NaN으로 반환한다.
# 모든 경우의 수를 다 볼 수 있지만 너무 데이터가 커져서 문제가 될 수 있음
pd.merge(df_left, df_right, how='outer')

,KEY,A,B,C,D
0,k0,a0,b0,NaN,NaN
1,k1,a1,b1,NaN,NaN
2,k2,a2,b2,c2,d2
3,k3,a3,b3,c3,d3
4,k4,NaN,NaN,c4,d4
5,k5,NaN,NaN,c5,d5


In [60]:
# 실습 
# 아래 2개의 DataFrame을 K1과 K2 컬럼으로 outer merge
df_left2 = pd.DataFrame({'K1': ['k0', 'k1', 'k2', 'k3'],
                      'A': ['a0', 'a1', 'a2', 'a3'],
                      'B': ['b0', 'b1', 'b2', 'b3']})
df_right2 = pd.DataFrame({'K2': ['k2', 'k3', 'k4', 'k5'],
                       'C': ['c2', 'c3', 'c4', 'c5'],
                       'D': ['d2', 'd3', 'd4', 'd5']})   

In [61]:
# 실습
pd.merge(df_left2, df_right2, left_on='K1', right_on='K2', how='outer')

,K1,A,B,K2,C,D
0,k0,a0,b0,NaN,NaN,NaN
1,k1,a1,b1,NaN,NaN,NaN
2,k2,a2,b2,k2,c2,d2
3,k3,a3,b3,k3,c3,d3
4,NaN,NaN,NaN,k4,c4,d4
5,NaN,NaN,NaN,k5,c5,d5


In [62]:
# 실제 데이터 실습
# 데이터 읽어오기
# 데이터 파일: 'data/movielens/users.dat'
# 컬럼명 = ['사용자아이디','성별','연령','직업','지역']
users = pd.read_csv("data/movielens/users.dat",
           sep="::", 
           names = ['사용자아이디','성별','연령','직업','지역'] )

users.head()

C:\Users\tkdal\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  import sys


,사용자아이디,성별,연령,직업,지역
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [63]:
# head() 
users.head()

,사용자아이디,성별,연령,직업,지역
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [64]:
# 평점 데이터 읽어오기
# 데이터 파일 : data/movielens/ratings.dat
# 컬럼명들은 ['사용자아이디', '영화아이디', '평점', '타임스탬프']
ratings =  pd.read_csv("data/movielens/ratings.dat",
           sep="::", 
           names = ['사용자아이디', '영화아이디', '평점', '타임스탬프'] )

C:\Users\tkdal\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [65]:
# head() 
ratings.head()

,사용자아이디,영화아이디,평점,타임스탬프
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [66]:
# 사용자와 평점 데이터 합치기(outer)
data2 = pd.merge(users, ratings, how='inner')
data2

,사용자아이디,성별,연령,직업,지역,영화아이디,평점,타임스탬프
0,1,F,1,10,48067,1193,5,978300760
1,1,F,1,10,48067,661,3,978302109
2,1,F,1,10,48067,914,3,978301968
3,1,F,1,10,48067,3408,4,978300275
4,1,F,1,10,48067,2355,5,978824291
...,...,...,...,...,...,...,...,...
1000204,6040,M,25,6,11106,1091,1,956716541
1000205,6040,M,25,6,11106,1094,5,956704887
1000206,6040,M,25,6,11106,562,5,956704746
1000207,6040,M,25,6,11106,1096,4,956715648


In [67]:
# 사용자와 평점 데이터 합치기(left)
pd.merge(users, ratings, how='left')

,사용자아이디,성별,연령,직업,지역,영화아이디,평점,타임스탬프
0,1,F,1,10,48067,1193,5,978300760
1,1,F,1,10,48067,661,3,978302109
2,1,F,1,10,48067,914,3,978301968
3,1,F,1,10,48067,3408,4,978300275
4,1,F,1,10,48067,2355,5,978824291
...,...,...,...,...,...,...,...,...
1000204,6040,M,25,6,11106,1091,1,956716541
1000205,6040,M,25,6,11106,1094,5,956704887
1000206,6040,M,25,6,11106,562,5,956704746
1000207,6040,M,25,6,11106,1096,4,956715648


In [68]:
users['성별'].value_counts()

M    4331
F    1709
Name: 성별, dtype: int64

In [71]:
# users에는 저장된 사용자지만, ratings에는 없는 사용자 찾기
# isnull()
data2['사용자아이디'].isnull().sum()

0

## concat
동일한 컬럼을 가진 데이터프레임들을 하나로 합치고 싶은 경우
- concat함수는 일단 무작정 이어 붙이기
- merge는 공통점이 있는걸로 합친다.

- concat은 합칠 애들을 [] 리스트 안에다가 묶어준다.
- axis는 축을 의미하는데 0이면 행으로 붙이고, 1이면 열로 합침

<img src="img/concat0예제.jpg" alt="csv파일예제" style="width: 600px;"/>

In [112]:
df1 = pd.DataFrame([[1,'aa'],[2,'bb']], 
                   columns = list('AB'))
df2 = pd.DataFrame([[3,'cc'],[4,'dd']], 
                   columns = list('AB'))

In [113]:
df1

,A,B
0,1,aa
1,2,bb


In [114]:
df2

,A,B
0,3,cc
1,4,dd


In [116]:
# df1과 df2 concat 수행하기
pd.concat([df1, df2], axis=0)

,A,B
0,1,aa
1,2,bb
0,3,cc
1,4,dd


In [118]:
# 샘플 데이터 생성
df1 = pd.DataFrame([[1,10],[2,20], [3,30]], 
                columns = list('AB'), 
                index = ['aa','bb', 'cc'])
df2 = pd.DataFrame([[3,'xx'],[4,'yy']], 
                columns = list('CD'), 
                index = ['aa','bb'])

In [119]:
df1

,A,B
aa,1,10
bb,2,20
cc,3,30


In [120]:
df2

,C,D
aa,3,xx
bb,4,yy


In [121]:
# concat with axis = 1 수행
pd.concat([df1, df2], axis=1)

C:\Users\tkdal\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,A,B,C,D
aa,1,10,3.0,xx
bb,2,20,4.0,yy
cc,3,30,NaN,NaN


# 2. 계층 색인 (다중 색인)
데이터가 복잡해지면 여러 개의 index와 column들이 다중으로 쌓이게 된다. 이 때 해당되는 정보들을 빼오기 위해 계층 색인(다중 색인)이라는 개념이 쓰이게 된다. <br>

- 2개 이상의 색인(인덱스)를 지정할 수 있다. <br>
- 차원이 높은 (고차원) 데이터를 낮은 차원의 형식으로 다룰 수 있게 해주는 기능

In [124]:
np.random.randint(50, 100)

74

In [125]:
np.random.randint(50, 100, (5, 4))

array([[67, 87, 75, 63],
       [58, 59, 70, 66],
       [55, 65, 97, 50],
       [68, 85, 74, 99],
       [79, 69, 69, 64]])

In [84]:
# 샘플 데이터 생성
np.random.seed(0)

df = pd.DataFrame(np.random.randint(50, 100, (5, 4)), 
                  columns=[[2016, 2016, 2017, 2017], ['영어','수학','영어','수학']], 
                  index = ['Kim','Park','Lee','Jung','Moon'])

- [2016, 2016, 2017, 2017] 의 의미는 더 상위의 컬럼이고 
- ["영어","수학" ....] 이런건 하위 컬럼들에 대한 정보
- randint(50, 100, (5, 4)) 의 의미는 50~100까지 값을 5행 4열짜리 배열에 넣어!라는 뜻<br>
= 즉 20개를 값으로 뽑겠다.
- random으로 뽑은걸 고정해주고 싶다면 np. random.seed를 해주면된다.

In [91]:
df

2016     2017    
       영어  수학   영어  수학
Kim    94  97   50  53
Park   53  89   59  69
Lee    71  86   73  56
Jung   74  74   62  51
Moon   88  89   73  96

In [129]:
# index확인
df.index[0]

'Kim'

In [130]:
# column 확인
df.columns[0][0]

2016

## xs() method - 열
xs는 계층색인이 되어있을때 하위레벨의 값을 불러오고 싶을때 사용한다.
- 한개의 column만 선택 가능 <--- 무조건 하나의 컬럼만을 불러옴
- axis=0, level=0 (default)
- axis=0(행), axis=1(열)
- level=0(상위레벨), level=1(하위레벨)

In [131]:
#2016년 영어, 수학 성적 조회
df.xs(2016, axis=1, level=0)

과목,영어,수학
학생명,,
Kim,94,97
Park,53,89
Lee,71,86
Jung,74,74
Moon,88,89


## 기존 index loc 방식 - 열

In [132]:
#2016년 영어, 수학 성적 조회
df[2016]

과목,영어,수학
학생명,,
Kim,94,97
Park,53,89
Lee,71,86
Jung,74,74
Moon,88,89


In [95]:
#2016년 영어 성적만 조회
#최상위 인덱스부터 명시하며, 인덱싱하고자하는 색인들을 튜플 형태로 정의
df[(2016, "영어")]

Kim     94
Park    53
Lee     71
Jung    74
Moon    88
Name: (2016, 영어), dtype: int32

In [96]:
# 2017년 수학 조회
df[(2017,"수학")]

Kim     53
Park    69
Lee     56
Jung    51
Moon    96
Name: (2017, 수학), dtype: int32

In [97]:
# 2017년 Kim의 수학 점수 조회
df.loc["Kim", (2017,"수학")]

53

## xs() method - 행 

In [98]:
# Kim의 성적만 선택
df.xs('Kim', axis=0)

2016  영어    94
      수학    97
2017  영어    50
      수학    53
Name: Kim, dtype: int32

## 기본 index 방식 - 행

In [99]:
# Kim의 성적만 선택
df.loc["Kim"]

2016  영어    94
      수학    97
2017  영어    50
      수학    53
Name: Kim, dtype: int32

In [100]:
# 실습 1 
# Kim, Park, Lee의 성적만 선택
df.loc[["Kim", "Park", "Lee"]]

2016     2017    
       영어  수학   영어  수학
Kim    94  97   50  53
Park   53  89   59  69
Lee    71  86   73  56

In [101]:
#  실습 2
# 'Kim', 'Lee'의 2016년 영어 성적 조회
df.loc[["Kim", "Lee"], (2016, "영어")]

Kim    94
Lee    71
Name: (2016, 영어), dtype: int32

- "영어"라는 컬럼 하나만 지정했기 때문에 xs함수를 써주면 훨씬 편하다.

In [133]:
# 실습 3
# 2016, 2017년도 영어 성적만 선택
df.xs("영어", axis=1, level=1)

연도,2016,2017
학생명,,
Kim,94,50
Park,53,59
Lee,71,73
Jung,74,62
Moon,88,73


In [134]:
# row index의 이름을 '학생명'이라고 정의하기
df.index.set_names("학생명", inplace=True)
df

연도   2016     2017    
과목     영어  수학   영어  수학
학생명                   
Kim    94  97   50  53
Park   53  89   59  69
Lee    71  86   73  56
Jung   74  74   62  51
Moon   88  89   73  96

In [135]:
# column index의 이름을 '연도', '과목'이라고 정의하기
df.columns.set_names(["연도", "과목"], inplace=True)
df

연도   2016     2017    
과목     영어  수학   영어  수학
학생명                   
Kim    94  97   50  53
Park   53  89   59  69
Lee    71  86   73  56
Jung   74  74   62  51
Moon   88  89   73  96

## swaplevel(index1, index2, axis)
index와 column의 순서(위치)를 변경해주고 싶을 때 사용한다
- index1과 index2의 위치를 변경함. 
- index1과 index2가 row index 경우, axis = 0
- index1과 index2가 column index 경우, axis = 1

In [105]:
# 년도와 과목의 위치를 변경
df.swaplevel("연도", "과목", axis=1)

과목,영어,수학,영어,수학
연도,2016,2016,2017,2017
학생명,,,,
Kim,94,97,50,53
Park,53,89,59,69
Lee,71,86,73,56
Jung,74,74,62,51
Moon,88,89,73,96


## stack(), unstack() 함수
- stack(level) : 컬럼 인덱스를 로우 인덱스로 옮길 때 사용.
- unstack(level): 로우 인덱스를 컬럼 인덱스로 옮길 때 사용.
- level 인자는 옮기고자 하는 인덱스의 위치를 표기함. 명시하지 않은 경우, 최하단의 인덱스를 이동시킴.
- level은 최상위가 0이고, 1씩 증가함

In [136]:
df

연도   2016     2017    
과목     영어  수학   영어  수학
학생명                   
Kim    94  97   50  53
Park   53  89   59  69
Lee    71  86   73  56
Jung   74  74   62  51
Moon   88  89   73  96

In [141]:
# 컬럼 인덱스 과목을 로우 인덱스로 변경하고 df2에 저장
df2 = df.stack(level=1)

In [142]:
df2

연도       2016  2017
학생명  과목            
Kim  수학    97    53
     영어    94    50
Park 수학    89    69
     영어    53    59
Lee  수학    86    56
     영어    71    73
Jung 수학    74    51
     영어    74    62
Moon 수학    89    96
     영어    88    73

In [107]:
# 실습1
# Kim의 성적만 선택
df2.loc["Kim"]

연도,2016,2017
과목,,
수학,97,53
영어,94,50


In [108]:
# 실습2
# Park의 수학 성적만 선택
df2.loc[("Park", "수학")]

연도
2016    89
2017    69
Name: (Park, 수학), dtype: int32

In [109]:
# 실습3
# 모든 학생들의 영어 성적만 선택 
df2.xs("영어",axis=0,level=1)

연도,2016,2017
학생명,,
Kim,94,50
Park,53,59
Lee,71,73
Jung,74,62
Moon,88,73


In [110]:
# 실습4
# Park 학생의 2016년 영어 성적만 출력
df2.loc[("Park","영어"), 2016]

53

In [111]:
# 실습5
# 학생들의 과목별 성적의 평균을 구해서, 새로운 컬럼 '평균'으로 저장
df2["평균2"] = df2.mean(axis=1)
df2

연도       2016  2017   평균2
학생명  과목                  
Kim  수학    97    53  75.0
     영어    94    50  72.0
Park 수학    89    69  79.0
     영어    53    59  56.0
Lee  수학    86    56  71.0
     영어    71    73  72.0
Jung 수학    74    51  62.5
     영어    74    62  68.0
Moon 수학    89    96  92.5
     영어    88    73  80.5

# 3. 통계함수와 데이터 정렬하기

## 주요 통계 함수 목록
<img src="img/통계함수.jpg" alt="통계함수" style="width: 400px;"/>

## 주요 함수 옵션
<img src="img/주요함수옵션.jpg" alt="주요옵션" style="width: 400px;"/>

In [143]:
# 샘플 데이터 생성
np.random.seed(321)

df = pd.DataFrame(np.random.randint(50, 100, (5, 4)), 
                  columns= [np.repeat([2016, 2017],2), ['영어', '수학']*2 ],
                  index = ['Kim','Park','Lee','Jung','Moon'])
df

2016     2017    
       영어  수학   영어  수학
Kim    76  81   91  58
Park   67  90   76  74
Lee    87  58   69  71
Jung   51  77   76  63
Moon   93  81   60  87

In [144]:
# index 이름으로 학생명, column 이름으로 연도와 과목 부여
df.index.set_names('학생명', inplace=True)
df.columns.set_names(['연도','과목'], inplace=True)
df

연도   2016     2017    
과목     영어  수학   영어  수학
학생명                   
Kim    76  81   91  58
Park   67  90   76  74
Lee    87  58   69  71
Jung   51  77   76  63
Moon   93  81   60  87

In [146]:
# np.nan은 "nan"값이 아니라, 빈 값이다.
np.nan

nan

In [147]:
# Kim 학생의 2016년 영어 성적을 NA로 처리하기
df.loc['Kim',(2016,'영어')]=np.nan

In [148]:
# 2016년 성적만 선택해서 df2016에 저장
df2016 = df[2016]
df2016

과목,영어,수학
학생명,,
Kim,NaN,81
Park,67.0,90
Lee,87.0,58
Jung,51.0,77
Moon,93.0,81


## 통계 함수

In [149]:
# count() - NaN 값을 제외한 데이터의 갯수 
df.count()

연도    과목
2016  영어    4
      수학    5
2017  영어    5
      수학    5
dtype: int64

In [163]:
# 원래 값으로 되돌려 놓기
df.loc['Kim',(2016,'영어')] = 76
df2016 = df[2016]
df2016

과목,영어,수학
학생명,,
Kim,76.0,81
Park,67.0,90
Lee,87.0,58
Jung,51.0,77
Moon,93.0,81


In [164]:
# describe() - 각 컬럼에 대한 요약 통계량을 확인해주는 함수
df.describe()

연도          2016                 2017           
과목            영어        수학         영어         수학
count   5.000000   5.00000   5.000000   5.000000
mean   74.800000  77.40000  74.400000  70.600000
std    16.649324  11.84483  11.371016  11.148991
min    51.000000  58.00000  60.000000  58.000000
25%    67.000000  77.00000  69.000000  63.000000
50%    76.000000  81.00000  76.000000  71.000000
75%    87.000000  81.00000  76.000000  74.000000
max    93.000000  90.00000  91.000000  87.000000

In [165]:
# sum() - 값의 합을 계산
# 기본적으로 함수는 각 컬럼에 있는 로우(row)들의 값들로 수행 (default: axis =0) 
df2016.sum()

과목
영어    374.0
수학    387.0
dtype: float64

In [166]:
# mean() - 값의 평균을 계산
# 각 학생들의 과목 평균 구하기
df2016.mean(axis=1)

학생명
Kim     78.5
Park    78.5
Lee     72.5
Jung    64.0
Moon    87.0
dtype: float64

In [167]:
# cumsum() - cumulative sum
df2016.cumsum(axis=0)

과목,영어,수학
학생명,,
Kim,76.0,81.0
Park,143.0,171.0
Lee,230.0,229.0
Jung,281.0,306.0
Moon,374.0,387.0


# 데이터 정렬하기

<img src="img/데이터프레임정렬.jpg" alt="DataFrame.sort_values" style="width: 600px;"/>
- 인덱스 정렬 : sort_index() - 인자는 sort_values()와 동일

## Series 정렬

In [168]:
# Series 정렬을 위한 샘플 데이터
sr = pd.Series({'b':3, 'c':5, 'a':2, 'f':1, 'e':7, 'd':np.nan})
sr

b    3.0
c    5.0
a    2.0
f    1.0
e    7.0
d    NaN
dtype: float64

#### index정렬 : sort_index()

In [169]:
# 오름차순 정렬, 기본값 = True
sr.sort_index(ascending='True')

a    2.0
b    3.0
c    5.0
d    NaN
e    7.0
f    1.0
dtype: float64

In [170]:
# 내림차순 정렬
sr.sort_index(ascending='False')

a    2.0
b    3.0
c    5.0
d    NaN
e    7.0
f    1.0
dtype: float64

#### value정렬 : sort_values()

In [171]:
# NA값을 맨 마지막으로 오름차순 정렬, 기본값 = True
sr.sort_values(ascending=True, na_position='last')

f    1.0
a    2.0
b    3.0
c    5.0
e    7.0
d    NaN
dtype: float64

In [19]:
# NA값을 맨 처음으로 내림차순 정렬
sr.sort_values(ascending=False, na_position='first')

d    NaN
e    7.0
c    5.0
b    3.0
a    2.0
f    1.0
dtype: float64

## DataFrame 정렬

In [172]:
# sample data 생성
dic = {'영어':[10, 60, 80, 30, 40, 50, 20, 70, 30, 90],
       '수학':[90, 80, 70, 60, 50, 50, 40, 30, 30, 20]}

df2 = pd.DataFrame(dic)
df2

,영어,수학
0,10,90
1,60,80
2,80,70
3,30,60
4,40,50
5,50,50
6,20,40
7,70,30
8,30,30
9,90,20


In [175]:
# row index 정렬 (default: axis=0) 
df2.sort_index(axis=0)

,영어,수학
0,10,90
1,60,80
2,80,70
3,30,60
4,40,50
5,50,50
6,20,40
7,70,30
8,30,30
9,90,20


In [22]:
# column index 정렬
df2.sort_index(axis=1)

,수학,영어
0,90,10
1,80,60
2,70,80
3,60,30
4,50,40
5,50,50
6,40,20
7,30,70
8,30,30
9,20,90


In [23]:
# 수학 성적으로 오름차순 정렬
df2.sort_values(by='수학', ascending=True)

,영어,수학
9,90,20
7,70,30
8,30,30
6,20,40
4,40,50
5,50,50
3,30,60
2,80,70
1,60,80
0,10,90


In [24]:
# 영어 성적으로 내림차순 정렬
df2.sort_values(by='영어', ascending=False)

,영어,수학
9,90,20
2,80,70
7,70,30
1,60,80
5,50,50
4,40,50
3,30,60
8,30,30
6,20,40
0,10,90


In [25]:
# column index 정렬하고 수학을 기준으로 내림차순 정렬
df2.sort_index(axis=1, inplace=True)
df2.sort_values(by='수학', ascending=False)

,수학,영어
0,90,10
1,80,60
2,70,80
3,60,30
4,50,40
5,50,50
6,40,20
7,30,70
8,30,30
9,20,90


In [176]:
# 2개의 기준으로 정렬
# 수학을 기준으로 내림차순 정렬하고 수학 점수가 같은 경우 영어 성적으로 내림차순 정렬
df2.sort_values(by=['수학', '영어'], ascending=[False, False])

,영어,수학
0,10,90
1,60,80
2,80,70
3,30,60
5,50,50
4,40,50
6,20,40
7,70,30
8,30,30
9,90,20


In [178]:
# 수학을 기준으로 내림차순 정렬하고 수학 점수가 같은 경우 영어 성적으로 오름차순 정렬
df2.sort_values(by=['수학','영어'], ascending=[False, True]) # [False, False]

,영어,수학
0,10,90
1,60,80
2,80,70
3,30,60
4,40,50
5,50,50
6,20,40
8,30,30
7,70,30
9,90,20


## 계층 색인 DataFrame 정렬

In [179]:
df

연도    2016     2017    
과목      영어  수학   영어  수학
학생명                    
Kim   76.0  81   91  58
Park  67.0  90   76  74
Lee   87.0  58   69  71
Jung  51.0  77   76  63
Moon  93.0  81   60  87

In [180]:
# 2017년 수학 성적 기준으로 내림차순 정렬
df.sort_values(by=(2017,'수학'), ascending=False)

연도    2016     2017    
과목      영어  수학   영어  수학
학생명                    
Moon  93.0  81   60  87
Park  67.0  90   76  74
Lee   87.0  58   69  71
Jung  51.0  77   76  63
Kim   76.0  81   91  58

In [181]:
# 2016년 평균을 구하고 2016년 평균을 기준으로 내림차순 정렬
df[(2016, '평균')] = df[2016].mean(axis=1)
df.sort_values(by=(2016,'평균'), ascending=False)

연도    2016     2017      2016
과목      영어  수학   영어  수학    평균
학생명                          
Moon  93.0  81   60  87  87.0
Kim   76.0  81   91  58  78.5
Park  67.0  90   76  74  78.5
Lee   87.0  58   69  71  72.5
Jung  51.0  77   76  63  64.0

# 집계(groupby, pivot table)

<img src="img/그룹분석_groupby.jpg" alt="그룹분석_groupby" style="width: 800px;"/>
<img src="img/그룹분석_pivottable.jpg" alt="그룹분석_groupby" style="width: 800px;"/>

In [182]:
# 실습 데이터 적재
data = pd.read_excel('data/인구수예제.xlsx')
data.head()

,도시,자치구,연도,남자인구,여자인구,총인구
0,서울,강남구,2013,73,92,165
1,서울,강남구,2014,139,55,194
2,서울,강남구,2015,123,83,206
3,서울,강남구,2016,147,150,297
4,서울,강남구,2017,57,133,190


## groupby
- df.groupby('묶고 싶은 기준 열')['연산 하고 싶은 열'].함수

In [33]:
# 연도별 총인구의 합
# 집단별로 묶고 싶은 기준 : '연도'
# 연산을 하고 싶은 column : '총인구'
# 사용할 함수 : 'sum()'
pd.DataFrame(data.groupby('연도')['총인구'].sum())

,총인구
연도,
2013,2130
2014,2256
2015,1848
2016,2045
2017,2128


In [34]:
# reset_index() : index의 값을 column으로 이동
pd.DataFrame(data.groupby('연도')['총인구'].sum()).reset_index()

,연도,총인구
0,2013,2130
1,2014,2256
2,2015,1848
3,2016,2045
4,2017,2128


In [184]:
# 자치구별 총인구의 평균
pd.DataFrame(data.groupby('자치구')['총인구'].mean())

,총인구
자치구,
강남구,210.4
도봉구,207.0
동래구,207.2
동작구,207.2
서대문구,206.2
송파구,194.8
수영구,212.2
영등포구,238.2
종로구,171.2


## pivot_table
pivot_table은 groupby와 다르게 자동적으로 DataFrame형태로 만들어준다.
- df.pivot_table(index='', aggfunc='', values=''), <br>
index = '묶고 싶은 기준열'<br>
aggfunc = 함수<br>
values = '연산 하고 싶은 열'<br>

In [185]:
# 연도별 총인구의 합
data.pivot_table(index='연도', aggfunc='sum', values='총인구')

,총인구
연도,
2013,2130
2014,2256
2015,1848
2016,2045
2017,2128


In [37]:
# 자치구별 총인구의 평균
data.pivot_table(index='자치구', aggfunc='mean', values='총인구')

,총인구
자치구,
강남구,210.4
도봉구,207.0
동래구,207.2
동작구,207.2
서대문구,206.2
송파구,194.8
수영구,212.2
영등포구,238.2
종로구,171.2


#### 실습

1) 연도별 남자인구, 여자인구, 총인구 수 합 구하기

In [38]:
# groupby
data.groupby('연도')[ ['남자인구', '여자인구', '총인구'] ].sum()

,남자인구,여자인구,총인구
연도,,,
2013,1099,1031,2130
2014,1196,1060,2256
2015,872,976,1848
2016,947,1098,2045
2017,1063,1065,2128


In [39]:
# pivot_table
data.pivot_table(index='연도', aggfunc='sum', values=['남자인구', '여자인구', '총인구'])

,남자인구,여자인구,총인구
연도,,,
2013,1099,1031,2130
2014,1196,1060,2256
2015,872,976,1848
2016,947,1098,2045
2017,1063,1065,2128


2) 도시/자치구별 평균 총인구수 구하기

In [45]:
# groupby
data.groupby( ['도시', '자치구'] )['총인구'].mean()

도시  자치구 
부산  동래구     207.2
    수영구     212.2
    해운대구    227.0
서울  강남구     210.4
    도봉구     207.0
    동작구     207.2
    서대문구    206.2
    송파구     194.8
    영등포구    238.2
    종로구     171.2
Name: 총인구, dtype: float64

In [41]:
# pivot_table
data.pivot_table(index=['도시', '자치구'], aggfunc='mean', values='총인구')

총인구
도시 자치구        
부산 동래구   207.2
   수영구   212.2
   해운대구  227.0
서울 강남구   210.4
   도봉구   207.0
   동작구   207.2
   서대문구  206.2
   송파구   194.8
   영등포구  238.2
   종로구   171.2

3) 연도별, 도시별 총인구수 합 구하기

In [190]:
# groupby
pd.DataFrame(data.groupby(['도시','연도'])['총인구'].sum()).unstack('도시')

총인구      
도시     부산    서울
연도             
2013  603  1527
2014  683  1573
2015  597  1251
2016  652  1393
2017  697  1431

In [187]:
# pivot_table
data.pivot_table(index='연도', columns='도시', aggfunc='sum', values='총인구')

도시,부산,서울
연도,,
2013,603,1527
2014,683,1573
2015,597,1251
2016,652,1393
2017,697,1431
